# Overview

This project demonstrates how to create two AI-powered agents using OpenAI's openai-agents SDK:

## Stock Analysis Agent

- Fetches live stock data from Yahoo Finance using WebSearchTool.
- Parses key stock metrics (closing price, 50-day SMA, 200-day SMA, volatility).
- Uses OpenAI’s GPT to analyze the data and generate investment insights.
- Visualizes insights using tables and bar charts.

## 1️.  Install Required Packages

In [14]:
%%capture
!pip3 install openai-agents

## 2️. Import Required Libraries

In [15]:
import openai
from agents import Agent, Runner, WebSearchTool
import os
import re
import pandas as pd
import matplotlib.pyplot as plt

## 3️. Set Up OpenAI API Key

In [17]:
openai_apikey = ""
os.environ["OPENAI_API_KEY"]=openai_apikey
openai.api_key = openai_apikey


## 4️. Create a Stock Analysis Agent Using WebSearchTool

In [23]:
# Create an agent with WebSearchTool enabled
stock_agent = Agent(
    name="StockAnalysisAgent",
    instructions="Fetch real-time stock data from Yahoo Finance, analyze trends, and provide insights.",
    tools=[WebSearchTool()]  # Enable web search capability
)

print(f" Agent created: {stock_agent.name}")

 Agent created: StockAnalysisAgent


## 5️. Fetch Stock Data Using WebSearchTool

In [24]:
async def get_stock_data_from_web(ticker):
    """Use WebSearchTool to fetch stock data from Yahoo Finance."""
    search_query = f"{ticker} stock price site:finance.yahoo.com"
    
    prompt = f"Search for '{search_query}' and extract the latest closing price, 50-day SMA, 200-day SMA, and volatility."

    # Run the web search tool
    response = await Runner.run(stock_agent, prompt)
    
    return response


## 6️. Extract Stock Information

In [25]:
def extract_stock_info(response_text):
    """Extract numerical stock values from web search results."""
    data_extracted = {
        "Metric": ["Last Closing Price", "50-day SMA", "200-day SMA", "Volatility"],
        "Value": []
    }
    
    # Use regex to extract values
    patterns = {
        "Last Closing Price": r"Last Closing Price: \$([\d,.]+)",
        "50-day SMA": r"50-day SMA: \$([\d,.]+)",
        "200-day SMA": r"200-day SMA: \$([\d,.]+|Not available)",
        "Volatility": r"Volatility: ([\d.]+)"
    }

    for metric, pattern in patterns.items():
        match = re.search(pattern, response_text)
        value = match.group(1) if match else "N/A"
        data_extracted["Value"].append(value)

    return pd.DataFrame(data_extracted)


## 7️⃣ Generate AI Insights on Stock Trends

In [28]:
ticker = "IBM"  # Example stock
response = await get_stock_data_from_web(ticker)
print(response)

RunResult(input="Search for 'IBM stock price site:finance.yahoo.com' and extract the latest closing price, 50-day SMA, 200-day SMA, and volatility.", new_items=[ToolCallItem(agent=Agent(name='StockAnalysisAgent', instructions='Fetch real-time stock data from Yahoo Finance, analyze trends, and provide insights.', handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[WebSearchTool(user_location=None, search_context_size='medium')], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None), raw_item=ResponseFunctionWebSearch(id='ws_67d49e813854819187a1a47c3dc5f1fa07b0e21157208d60', status='completed', type='web_search_call'), type='tool_call_item'), MessageOutputItem(agent=Agent(name='StockAnalysisAgent', instructions='Fetch real-time stock data from Yahoo Finance, analyze trends, and pr

In [32]:
response_text = response.final_output
print(response_text)

As of March 14, 2025, IBM's stock performance is as follows:

- **Latest Closing Price**: $217.70
- **Previous Close**: $213.89
- **Open**: $214.33
- **Day's Range**: $213.74 - $217.85
- **52-Week Range**: $135.87 - $218.84
- **Volume**: 3,875,121
- **Average Volume**: 3,637,512
- **Market Cap**: $200.534 billion
- **Beta (5Y Monthly)**: 0.69
- **PE Ratio (TTM)**: 24.03
- **EPS (TTM)**: $9.06
- **Forward Dividend & Yield**: $6.68 (3.07%)
- **Ex-Dividend Date**: August 9, 2024
- **1-Year Target Estimate**: $194.43

Please note that specific values for the 50-day and 200-day Simple Moving Averages (SMA) and volatility are not provided in the available data. For the most accurate and up-to-date information, I recommend visiting IBM's profile on Yahoo Finance: ([finance.yahoo.com](https://finance.yahoo.com/quote/IBM/?ncid=yahooproperties_stockrecom_g40boan2td8&p=IBM&utm_source=openai)). 


In [34]:
 # Construct AI analysis prompt
prompt = f"""
Analyze the stock {ticker} based on the latest available data:
{response_text}

Provide insights on the stock's short-term and long-term trends.
"""

# Run the AI analysis
insights = await Runner.run(stock_agent, prompt)
print(f" AI Insights for {ticker}:\n{insights}")

 AI Insights for IBM:
RunResult(input="\nAnalyze the stock IBM based on the latest available data:\nAs of March 14, 2025, IBM's stock performance is as follows:\n\n- **Latest Closing Price**: $217.70\n- **Previous Close**: $213.89\n- **Open**: $214.33\n- **Day's Range**: $213.74 - $217.85\n- **52-Week Range**: $135.87 - $218.84\n- **Volume**: 3,875,121\n- **Average Volume**: 3,637,512\n- **Market Cap**: $200.534 billion\n- **Beta (5Y Monthly)**: 0.69\n- **PE Ratio (TTM)**: 24.03\n- **EPS (TTM)**: $9.06\n- **Forward Dividend & Yield**: $6.68 (3.07%)\n- **Ex-Dividend Date**: August 9, 2024\n- **1-Year Target Estimate**: $194.43\n\nPlease note that specific values for the 50-day and 200-day Simple Moving Averages (SMA) and volatility are not provided in the available data. For the most accurate and up-to-date information, I recommend visiting IBM's profile on Yahoo Finance: ([finance.yahoo.com](https://finance.yahoo.com/quote/IBM/?ncid=yahooproperties_stockrecom_g40boan2td8&p=IBM&utm_sour

In [35]:
print(insights.final_output)

Based on the provided data, let's analyze IBM's stock performance:

### Short-Term Analysis

1. **Recent Performance**: 
   - The stock closed at $217.70, up from the previous close of $213.89, indicating positive momentum in the short term.
   - The opening price ($214.33) was below the closing price, suggesting a rise throughout the day.

2. **Day's Range**:
   - The day's range ($213.74 - $217.85) shows some volatility, with a peak close to the 52-week high.

3. **Volume**:
   - The trading volume (3,875,121) is slightly above the average volume (3,637,512), suggesting increased investor interest.

### Long-Term Analysis

1. **52-Week Range**:
   - The stock is trading near the upper end of its 52-week range ($135.87 - $218.84), indicating strong performance over the year.

2. **Market Cap**:
   - A market cap of $200.534 billion reflects IBM’s position as a well-established company, attracting long-term investors.

3. **Beta**:
   - A beta of 0.69 suggests IBM is less volatile than